<a href="https://colab.research.google.com/github/EddyGiusepe/Studying_spaCy_NER/blob/main/Custom_NER_with_spaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align="center">Custom NER with spaCy v3</h2>



Data scientist.: Dr.Eddy Giusepe Chirinos Isidro

In [1]:
! pip install -U spacy -q

     |████████████████████████████████| 6.2 MB 7.5 MB/s 
     |████████████████████████████████| 653 kB 69.5 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 457 kB 74.4 MB/s 
     |████████████████████████████████| 181 kB 68.5 MB/s 
     |████████████████████████████████| 10.1 MB 73.7 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [2]:

!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.3.0                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.13                        
Pipelines                                      



In [3]:
# Aqui você pode baixar o arquivo .json (dados de train)

!wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

--2022-05-12 22:48:59--  https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829 (2.8K) [text/plain]
Saving to: ‘pvr_training_data.json’

pvr_training_data.j 100%[===================>]   2.76K  --.-KB/s    in 0s      

2022-05-12 22:48:59 (50.4 MB/s) - ‘pvr_training_data.json’ saved [2829/2829]



In [4]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [6]:
import json
f = open('/content/training_data.json')
TRAIN_DATA = json.load(f)

In [9]:
TRAIN_DATA


{'annotations': [['The share price of PVR rose over 7 percent on Wednesday after the multiplex chain said that it has reduced losses in Q2 despite nil revenue from the core movie exhibition business.',
   {'entities': [[19, 22, 'COMPANY'],
     [33, 42, 'PERCENTAGE'],
     [46, 55, 'WEEKDAY']]}],
  ['The company managed to get rent waivers from most landlords, CFO Nitin Sood said in an interview to CNBC-TV18. “The big focus for us right now as revenues have been nil is to really reduce our fixed cost and we have managed to do that, ” he added.',
   {'entities': [[61, 64, 'ROLE'], [65, 75, 'NAME']]}],
  ['Sood further said that they have brought down the fixed cost down by almost 75-80 percent.',
   {'entities': [[0, 4, 'NAME'], [76, 89, 'PERCENTAGE']]}],
  ["The stock rose as much as 7.6 percent to the day's high of Rs 1,186.85 per share on the BSE.",
   {'entities': [[26, 37, 'PERCENTAGE'],
     [59, 70, 'PERCENTAGE'],
     [88, 91, 'COMPANY']]}],
  ['Meanwhile, for the September quar

In [11]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # salvamos o objeto docbin

100%|██████████| 12/12 [00:00<00:00, 4157.24it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [12]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-05-12 23:05:25,170] [INFO] Set up nlp object from config
[2022-05-12 23:05:25,179] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-05-12 23:05:25,183] [INFO] Created vocabulary
[2022-05-12 23:05:25,184] [INFO] Finished initializing nlp object
[2022-05-12 23:05:25,322] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.83    0.00    0.00    0.00    0.00
 23     200        486.52   1668.35  100.00  100.00  100.00    1.00
 52     400          0.00      0.00  100.00  100.00  100.00    1.00

In [14]:
# Carregamos nosso MODELO

nlp_ner = spacy.load("/content/model-best")

In [15]:
doc = nlp_ner('''The global cryptocurrency market capitalisation fell 4.49 percent over the last 24 hours to $1.75 trillion, while trading volumes declined 3.43 percent to $83.23 billion during the period.

The total volume in the decentralised finance (DeFi) space stood at $14.08 billion, around 16.92 percent of the 24-hour cryptocurrency trading volume. The total volume in stablecoins stood at $69.85 billion, around 83.92 percent of the 24-hour cryptocurrency trading volume.

Bitcoin's market dominance fell 0.57 percent to 42.44 percent and the currency was trading at $39,047.24 on the morning of March 5.

In rupee terms, Bitcoin declined 4.43 percent to trade at Rs 31,20,419 while Ethereum fell 2.6 percent to Rs 2,09,584.5

Cardano was down 2.98 percent to Rs 66.86 and Avalanche fell 2.01 percent to Rs 6,003.3. Polkadot was down 3.65 percent to Rs 1,319.01 and Litecoin fell 4.3 percent to Rs 8,058 in the last 24 hours. Tether was up 0.86 percent to Rs 79.52.

RELATED STORIES
  Campbell Wilson: 10 things to know about the new Air India boss 
  Crypto crash: Twitter's answer to buy bitcoin 'dips' whose price goes 'dipping' 
  Bitcoin hits 16-month low; Crypto crash decoded 
Memecoin SHIB fell 1.96 percent while Dogecoin was down by 1.27 percent to trade at Rs 9.79. Terra (LUNA) fell 10.78 percent to Rs 6,549.03.

According to a story published in the
Financial Times
, the Swiss federal government was planning to freeze crypto assets owned by Russian citizens and businesses held within Switzerland borders. The freeze would coincide with sanctions already imposed by the European Union in response to Russia's invasion of Ukraine.

Swiss Finance Minister Guy Parmelin says 223 Russians, including close associates of President Vladimir Putin, have had their bank accounts and physical assets frozen by Switzerland in the past week. The cryptocurrency prohibitions are an additional penalty beyond EU sanctions.

A senior official with the finance minister told the
Financial Times
 that freezing the crypto assets was necessary because Switzerland wanted to protect the integrity of its blockchain industry. According to a report by CV VC, a Swiss venture capitalist firm, as of December 2021, around 1,128 blockchain companies called Switzerland or neighboring principality Liechtenstein home.

As of 9:10 am, these were the prices of various cryptocurrencies in the Indian market (Data from WazirX)''')

In [17]:
spacy.displacy.render(doc, style='ent', jupyter=True) # display in Jupyter